In [161]:
import re 
import PyPDF2 as pdf2
import pandas as pd

In [162]:
header = pdf2.PdfReader('RAZÃO DE JANEIRO A OUTUBRO 2023.pdf')

In [163]:
for page in range(len(header.pages)):
    print(header.pages[page].extract_text())
    

Livro Caixa da Atividade Rural (Lei nº 9.250/95, art. 18)
TERMO DE ABERTURA
Este livro contém 54 folhas numeradas sequencialmente de 1 a 54 e servirá para o lançamento das receitas e
despesas, durante o ano-calendário de 2023, relativos à atividade rural do contribuinte abaixo identificado:BRASIL
Primeiro lançamento efetuado em: 1 de Janeiro de 2023
25/11/2023   08:10:28
000.854.898-67 CPF:
Nome:
Endereço:
Complemento:
Município:
DDD/Telefone:Nr:
Bairro:
U.F.:CEP:ANTONIO NELSON STUPELLO SANDOVAL
RUA TORRES HOMEM 331
AP 141
AraçatubaCENTRO
SP16010-360
(18) 36230122
Nome Moeda: Real
Inscrição Estadual / Municipal CIB(Nirf) Nome do Imóvel e Município
413057792112/SP FAZENDA FORMOSA - LAVINIA 0234439-4
285658735/MS FAZENDA PARAISO - AGUA CLARA 0234440-8
413057950118/SP FAZENDA FORMOSINHA - LAVINIA 1340359-1
Araçatuba, 1 de Janeiro de 2023
Página: 1
ANTONIO NELSON STUPELLO SANDOVALLivro Caixa da Atividade Rural
25/11/2023   08:10:28BRASIL
Conta Entrada Data Saída Saldo
01/01/2023 2.23.004 -

In [164]:
print(header.pages[1].extract_text())

ANTONIO NELSON STUPELLO SANDOVALLivro Caixa da Atividade Rural
25/11/2023   08:10:28BRASIL
Conta Entrada Data Saída Saldo
01/01/2023 2.23.004 - OUTROS CUSTOS DA
PECUARIA
Histórico: VR. PGTO NFE-3499 AUTO POSTO BENEDETTI DE ARACATUBA LTDA376,98 -376,98
04/01/2023 2.23.004 - OUTROS CUSTOS DA
PECUARIA
Histórico: VR. PGTO NFE-3713 AUTO POSTO BENEDETTI DE ARACATUBA LTDA391,53 -768,51
04/01/2023 2.23.004 - OUTROS CUSTOS DA
PECUARIA
Histórico: VR. PGTO ELEKTRO REF.12/2022137,56 -906,07
04/01/2023 2.23.004 - OUTROS CUSTOS DA
PECUARIA
Histórico: VR. PGTO NFE-3499 AUTO POSTO BENEDETTI DE ARACATUBA LTDA376,98 -1.283,05
05/01/2023 2.23.004 - OUTROS CUSTOS DA
PECUARIA
Histórico: VR. PGTO BOLETO 58272 NOVA GERACAO AUTO PECAS EIRELI ME125,00 -1.408,05
05/01/2023 2.23.004 - OUTROS CUSTOS DA
PECUARIA
Histórico: VR. PGTO BOLETO 13191 PREVINE ENG E SEG DO TRABALHO200,00 -1.608,05
05/01/2023 2.23.004 - OUTROS CUSTOS DA
PECUARIA
Histórico: VR. PGTO NFE-36565 BUENDIA PRODUTOS VETERINARIOS EIRELLI126,00 -1.7

In [165]:
re.findall('VR. PGTO .+', header.pages[1].extract_text())

['VR. PGTO NFE-3499 AUTO POSTO BENEDETTI DE ARACATUBA LTDA376,98 -376,98',
 'VR. PGTO NFE-3713 AUTO POSTO BENEDETTI DE ARACATUBA LTDA391,53 -768,51',
 'VR. PGTO ELEKTRO REF.12/2022137,56 -906,07',
 'VR. PGTO NFE-3499 AUTO POSTO BENEDETTI DE ARACATUBA LTDA376,98 -1.283,05',
 'VR. PGTO BOLETO 58272 NOVA GERACAO AUTO PECAS EIRELI ME125,00 -1.408,05',
 'VR. PGTO BOLETO 13191 PREVINE ENG E SEG DO TRABALHO200,00 -1.608,05',
 'VR. PGTO NFE-36565 BUENDIA PRODUTOS VETERINARIOS EIRELLI126,00 -1.734,05',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - ANTENOR DA SILVA NUNES2.418,23 -4.152,28',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - WESLEY HENRIQUE DA SILVA CARMO1.631,14 -5.783,42',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - MARIA LUIZA VIEIRA MACEDO1.541,28 -7.324,70',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - HELTON CANDIDO DA CRUZ2.814,11 -10.138,81',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - RUAN FEITOSA DOS REIS1.533,33 -11.672,14',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  

In [166]:
fulltext = ''
for page in range(len(header.pages)):
    fulltext += header.pages[page].extract_text()+'\n'

In [167]:
fulltext

"Livro Caixa da Atividade Rural (Lei nº 9.250/95, art. 18)\nTERMO DE ABERTURA\nEste livro contém 54 folhas numeradas sequencialmente de 1 a 54 e servirá para o lançamento das receitas e\ndespesas, durante o ano-calendário de 2023, relativos à atividade rural do contribuinte abaixo identificado:BRASIL\nPrimeiro lançamento efetuado em: 1 de Janeiro de 2023\n25/11/2023   08:10:28\n000.854.898-67 CPF:\nNome:\nEndereço:\nComplemento:\nMunicípio:\nDDD/Telefone:Nr:\nBairro:\nU.F.:CEP:ANTONIO NELSON STUPELLO SANDOVAL\nRUA TORRES HOMEM 331\nAP 141\nAraçatubaCENTRO\nSP16010-360\n(18) 36230122\nNome Moeda: Real\nInscrição Estadual / Municipal CIB(Nirf) Nome do Imóvel e Município\n413057792112/SP FAZENDA FORMOSA - LAVINIA 0234439-4\n285658735/MS FAZENDA PARAISO - AGUA CLARA 0234440-8\n413057950118/SP FAZENDA FORMOSINHA - LAVINIA 1340359-1\nAraçatuba, 1 de Janeiro de 2023\nPágina: 1\nANTONIO NELSON STUPELLO SANDOVALLivro Caixa da Atividade Rural\n25/11/2023   08:10:28BRASIL\nConta Entrada Data Saíd

In [168]:

datacodigo = re.findall('\d\d/\d\d/\d\d\d\d \d.\d\d.\d\d\d', fulltext)
print(datacodigo)


['01/01/2023 2.23.004', '04/01/2023 2.23.004', '04/01/2023 2.23.004', '04/01/2023 2.23.004', '05/01/2023 2.23.004', '05/01/2023 2.23.004', '05/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '06/01/2023 2.23.004', '07/01/2023 2.23.004', '07/01/2023 2.23.004', '07/01/2023 2.23.004', '07/01/2023 2.23.004', '09/01/2023 2.23.004', '10/01/2023 2.23.004', '10/01/2023 2.23.004', '10/01/2023 2.23.004', '10/01/2023 2.23.004', '10/01/2023 2.23.004', '10/01/2023 2.23.004', '11/01/2023 2.23.004', '11/01/2023 2.23.004', '12/01/2023 2.23.004', '13/01/2023 2.23.004', '16/01/2023 2.23.004', '16/01/2023 2.23.004', '16/01/2023 2.23.004', '16/01/2023 2.23.004', '16/01/2023 2.23.004', '16/01/2023 2.23.004', '16/01/2023 2.23.004', '16/01/202

In [169]:
for linha in range(len(datacodigo)):
    datacodigo[linha] = datacodigo[linha].replace('\n', '')

In [170]:
data = []
codigo = []
for linha in range(len(datacodigo)):
    if re.search('DA$', datacodigo[linha]):
        datacodigo[linha] += ' PECUARIA'
    data.append(re.search('\d\d/\d\d/\d\d\d\d', datacodigo[linha]).group())
    codigo.append(re.search('\d\.\d\d\.\d\d\d', datacodigo[linha]).group())

In [171]:
print(len(data))
print(len(codigo))

729
729


In [260]:
historicovalor = re.findall('Histórico: .+', fulltext)


In [270]:
historicovalor[141] = historicovalor[141] + '143,50 -695.820,58'
historicovalor[186] = historicovalor[186] + '180,00 -751.885,89'
historicovalor[271] = historicovalor[271] + '370,00 -71.243,00'
historicovalor[305] = historicovalor[305] + '588135820.790,00 -324.856,46'
historicovalor[367] = historicovalor[367] + '270,08 -646.138,04'
historicovalor[413] = historicovalor[413] + '1.761,79 -821.371,79'
historicovalor[465] = historicovalor[465] + '79,90 -932.663,86'
historicovalor[626] = historicovalor[626] + '1.200,01 -1.149.711,66'
historicovalor[726] = historicovalor[726] + 'CONF.NFP-19125.500,00 -1.266.140,07'

In [269]:
historicovalor[303]

'Histórico: VR. COMPRA DE 39 BEZERROS DE REUMILTO PEREIRA CUNHA, CONF.NFe-587995397.500,00 -249.066,46'

In [276]:
numerro = 0
saldo = []
for linha in range(len(historicovalor)):
    try: saldo.append(re.search(' -\d+\.?\d+\.?\d+,\d\d', historicovalor[linha]).group())
    except: 
        # historicovalor[linha] += ' ' + valorerro[numerro]
        # numerro += 1
        print(historicovalor[linha])
        print(linha)
        # saldo.append(re.search(' -?\d+\.?\d+\.?\d+,\d\d', historicovalor[linha]).group())
        


In [277]:
for item in range(len(saldo)):
    saldo[item] = saldo[item].replace(' ', '')
    saldo[item] = saldo[item].replace('.', '')
    saldo[item] = saldo[item].replace(',', '.')
    saldo[item] = float(saldo[item])

In [278]:
saldo[613]

-1115145.84

In [289]:
valor = []

valor.append(str(round(saldo[0],2)))
valor[0] = valor[0].replace('-', '')
valor[0] = valor[0].replace('.', ',')
if not re.search('\,\d\d',valor[0]):
    valor[0] += '0'
if len(valor[0]) >= 7:
    valor[0] = valor[0][:-6] + '.' + valor[0][-6:] 
historico = []
historico.append(re.search('.+'+valor[0], historicovalor[0]).group().replace(valor[0], ''))
historico[0] = historico[0][11:-2]
for item in range(1, len(saldo)):
    valor.append(str(round(saldo[item] - saldo[item-1],2)))
    valor[item] = valor[item].replace('-', '')
    valor[item] = valor[item].replace('.', ',')
    if not re.search('\,\d\d',valor[item]):
        valor[item] += '0'
    if len(valor[item]) >= 7:
        valor[item] = valor[item][:-6] + '.' + valor[item][-6:] 
    try: 
        historico.append(re.search('.+'+valor[item], historicovalor[item]).group().replace(valor[item], '')[11:])
        # print(historicovalor[item])
        # print(valor[item])
        # print(item)
    except:
        # print(historicovalor[item-1])
        # print(valor[item-1])
        # print(item-1)
        print(historicovalor[item])
        print(valor[item])
        print(item)

In [290]:
historico

['VR. PGTO NFE-3499 AUTO POSTO BENEDETTI DE ARACATUBA LTDA',
 'VR. PGTO NFE-3713 AUTO POSTO BENEDETTI DE ARACATUBA LTDA',
 'VR. PGTO ELEKTRO REF.12/2022',
 'VR. PGTO NFE-3499 AUTO POSTO BENEDETTI DE ARACATUBA LTDA',
 'VR. PGTO BOLETO 58272 NOVA GERACAO AUTO PECAS EIRELI ME',
 'VR. PGTO BOLETO 13191 PREVINE ENG E SEG DO TRABALHO',
 'VR. PGTO NFE-36565 BUENDIA PRODUTOS VETERINARIOS EIRELLI',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - ANTENOR DA SILVA NUNES',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - WESLEY HENRIQUE DA SILVA CARMO',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - MARIA LUIZA VIEIRA MACEDO',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - HELTON CANDIDO DA CRUZ',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - RUAN FEITOSA DOS REIS',
 'VR. PGTO SALARIO CONF.RECIBO REF.12/2022  - MATHEUS DIAS DE OLIVEIRA',
 'VR. PGTO NFE-233789 CAMDA-ARACATUBA',
 'VR. PGTO NFE-46519 ARACAPROLAB PRODUTOS PCARA LABORATORIO EIRELI ME',
 'VR. PGTO NFE-259848 COOPERCITRUS COOP DE PRODUTORES RUR

In [87]:
valor

['376,98',
 '391,53',
 '137,56',
 '376,98',
 '125,00',
 '200,00',
 '126,00',
 '2.418,23',
 '1.631,14',
 '1.541,28',
 '2.814,11',
 '1.533,33',
 '1.463,63',
 '64,90']

In [291]:
df = pd.DataFrame({'Data': data, 'Codigo' : codigo, 'Historico': historico, 'Valor': valor})

In [281]:
df

,Data,Codigo,Historico,Valor
0,01/01/2023,2.23.004,Histórico: VR. PGTO NFE-3499 AUTO POSTO BENEDE...,"376,98"
1,04/01/2023,2.23.004,Histórico: VR. PGTO NFE-3713 AUTO POSTO BENEDE...,"391,53"
2,04/01/2023,2.23.004,Histórico: VR. PGTO ELEKTRO REF.12/2022,"137,56"
3,04/01/2023,2.23.004,Histórico: VR. PGTO NFE-3499 AUTO POSTO BENEDE...,"376,98"
4,05/01/2023,2.23.004,Histórico: VR. PGTO BOLETO 58272 NOVA GERACAO ...,"125,00"
...,...,...,...,...
724,27/10/2023,2.23.004,Histórico: VR. PGTO NFE-186832 J DIONISIO VEIC...,"1.868,39"
725,27/10/2023,2.23.004,Histórico: VR. PGTO NFSe-105083 J DIONÍSIO VEI...,"731,61"
726,27/10/2023,1.15.001,"Histórico: VR, VENDA DE 14 BEZERRAS DE 9 A 12 ...","25.500,00"
727,30/10/2023,2.23.004,Histórico: VR. PGTO BOLETO REF.NFE-80825 HERBI...,"989,28"


In [294]:
df.to_csv('out.csv', sep=';', encoding='utf-8-sig')